# Keras 신경망 구성
  
###### 2024-2-25 KH.LIM @Cheongju MyHappyHome
XOR 연산을 하는 신경망을 만든다. 
1. 케라스를 이용해서 만들기  
2. 파이썬으로 Neural Network 구성  
  
###### 신경망 교과서 참조
###### over96@naver.com

In [3]:
!pip install keras

Defaulting to user installation because normal site-packages is not writeable


## 1. 케라스를 이용해서 만들기

In [ ]:


# 비어있는 Sequential 모델 만들기

from keras.models import Sequential
model = Sequential()

In [45]:
# 레이어 추가하기

from keras.layers import Dense
# 레이어 1 
# input_dim 은 입력요소(차원)의 갯수, units은 출력차원의 수
model.add(Dense(units=4, activation='sigmoid', input_dim=3))
# 출력 레이어
model.add(Dense(units=1, activation='sigmoid'))

In [46]:
# 모델 아키텍처 검토하기
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 4)                 16        
                                                                 
 dense_24 (Dense)            (None, 1)                 5         
                                                                 
Total params: 21 (84.00 Byte)
Trainable params: 21 (84.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


Param # 은 모델이 훈련해야 할 가중치와 편향의 개수

In [47]:
# 모델 컴파일 및 훈련

from keras import optimizers
sgd = optimizers.SGD(learning_rate=1)
model.compile(loss='mean_squared_error', optimizer=sgd)

In [48]:
# 학습데이터는 XOR 연산을 위한 데이터
import numpy as np

# 결과를 재현할 수 있도록 랜덤시드 고정
np.random.seed(9)
X = np.array([[0,0,1], [0,1,1], [1,0,1], [1,1,1]])
y = np.array([[0],[1],[1],[0]])

In [49]:
# 반복횟수 결정
model.fit(X,y, epochs=1500, verbose=False)

In [50]:
# 예측값
print(model.predict(X))

1/1 [==============================] - 0s 57ms/step
[[0.05001061]
 [0.95613647]
 [0.93248785]
 [0.07582481]]


  
### 레이어 확장해보기

In [52]:
# 비어있는 Sequential 모델 만들기

from keras.models import Sequential
model = Sequential()

In [53]:
# 레이어 추가하기

from keras.layers import Dense
# 레이어 1 
model.add(Dense(units=4, activation='sigmoid', input_dim=3))
# 레이어 2 (추가)
model.add(Dense(units=4, activation='sigmoid', input_dim=4))


# 출력 레이어
model.add(Dense(units=1, activation='sigmoid'))

In [54]:
# 모델 아키텍처 검토하기
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 4)                 16        
                                                                 
 dense_29 (Dense)            (None, 4)                 20        
                                                                 
 dense_30 (Dense)            (None, 1)                 5         
                                                                 
Total params: 41 (164.00 Byte)
Trainable params: 41 (164.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


Param # 은 모델이 훈련해야 할 가중치와 편향의 개수

In [55]:
# 모델 컴파일 및 훈련

from keras import optimizers
sgd = optimizers.SGD(learning_rate=1)
model.compile(loss='mean_squared_error', optimizer=sgd)

In [56]:
# 학습데이터는 XOR 연산을 위한 데이터
import numpy as np

# 결과를 재현할 수 있도록 랜덤시드 고정
np.random.seed(9)
X = np.array([[0,0,1], [0,1,1], [1,0,1], [1,1,1]])
y = np.array([[0],[1],[1],[0]])

In [57]:
# 반복횟수 결정
model.fit(X,y, epochs=1500, verbose=False)

In [58]:
# 예측값
print(model.predict(X))

1/1 [==============================] - 0s 62ms/step
[[0.11487298]
 [0.81290907]
 [0.88001204]
 [0.18288772]]


-----  

## 2. 파이선을 이용해 Neural Network 만들기

In [1]:

import numpy as np

# 시그모이드 함수 정의
def sigmoid(x):
    return 1.0/(1+ np.exp(-x))

# 시그모이드 함수 도함수(다소 이해 안 됨)
def sigmoid_derivative(x):
    return x * (1.0 - x)

# 은닉층1개인 신경망 구성
class NeuralNetwork:
    def __init__(self, x, y):
        self.input      = x
        self.weights1   = np.random.rand(self.input.shape[1],2)  #가중치의 크기는 입력값의 '열'값과 은닉층의 '행'값으로 결정. 이는 입력데이터를 참고하면 이해됨.
        self.weights2   = np.random.rand(2,1)                 
        self.y          = y
        self.output     = np.zeros(self.y.shape)

        #순전파 과정
    def feedforward(self):
        self.layer1 = sigmoid(np.dot(self.input, self.weights1))
        self.output = sigmoid(np.dot(self.layer1, self.weights2))

        #역전파 과정
    def backprop(self):
        # weight2와 weights1에 따른 손실 함수의 미분을 찾기 위해 연쇄법칙을 활용한다
        # T는 numpy 전치메서드
        d_weights2 = np.dot(self.layer1.T, (2*(self.y - self.output) * sigmoid_derivative(self.output)))
        d_weights1 = np.dot(self.input.T,  (np.dot(2*(self.y - self.output) * sigmoid_derivative(self.output), self.weights2.T) * sigmoid_derivative(self.layer1)))

        # 손실 함수의 미분 값을 사용해 가중치를 갱신한다
        self.weights1 += d_weights1
        self.weights2 += d_weights2


if __name__ == "__main__":
    X = np.array([[0,0,1],
                  [0,1,1],
                  [1,0,1],
                  [1,1,1]])
    y = np.array([[0],[1],[1],[0]])
    nn = NeuralNetwork(X,y)

    for i in range(3000):
        nn.feedforward()
        nn.backprop()

    print(nn.output)

[[0.04290954]
 [0.96855834]
 [0.96855835]
 [0.03951073]]
